# RecSys Model Ensemble (미완성)

##### Using Prediction Scores

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def ensemble_recommendations(folder_path, final_K, scaling_method='minmax', weights=None):
    # 주어진 폴더에서 .csv 화일 목록 가져오기
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # 각 화일을 데이터프레임으로 읽어오기
    dfs = [pd.read_csv(os.path.join(folder_path, csv_file)) for csv_file in csv_files]
    
    # 가중치가 주어지지 않은 경우, 모든 알고리즘에 동일한 가중치 (1) 부여
    if weights is None:
        weights = [1] * len(dfs)
    
    # 스케일러 초기화
    if scaling_method == 'minmax':
        scaler = MinMaxScaler()
    elif scaling_method == 'standard':
        scaler = StandardScaler()
    
    # 모든 데이터프레임들을 하나로 합치기 전에 각 데이터프레임마다 prediction 값을 정규화
    normalized_dfs = []
    for i, df in enumerate(dfs):
        df['prediction'] = scaler.fit_transform(df['prediction'].values.reshape(-1, 1)).flatten()
        # 가중치 적용
        df['prediction'] *= weights[i]
        normalized_dfs.append(df)

    # 정규화 및 가중치 적용된 데이터프레임들을 하나로 합치기
    combined_df = pd.concat(normalized_dfs, ignore_index=True)
    
    # resume_seq와 recruitment_seq를 기준으로 prediction 값을 그룹화하고 평균 계산
    averaged_predictions = combined_df.groupby(['resume_seq', 'recruitment_seq']).prediction.mean().reset_index()
    
    # 평균 점수를 기준으로 내림차순 정렬
    top_k = (
        averaged_predictions
        .groupby('resume_seq')
        .apply(lambda x: x.nlargest(final_K, 'prediction'))
        .reset_index(drop=True)
    )
    
    return top_k

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


##### Using Ranks

In [5]:
import os
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, StandardScaler

def ensemble_recommendations(folder_path, final_K, scaling_method='minmax', weights=None):
    # 주어진 폴더에서 .csv 화일 목록 가져오기
    csv_files = [f for f in os.listdir(folder_path) if f.endswith('.csv')]
    
    # 각 화일을 데이터프레임으로 읽어오기
    dfs = [pd.read_csv(os.path.join(folder_path, csv_file)) for csv_file in csv_files]
    
    # 가중치가 주어지지 않은 경우, 모든 알고리즘에 동일한 가중치 (1) 부여
    if weights is None:
        weights = [1] * len(dfs)
    
    # 모든 데이터프레임에서 prediction 값을 기반으로 순위를 계산하고 저장
    ranked_dfs = []
    for i, df in enumerate(dfs):
        df['prediction'] = df.groupby('resume_seq')['prediction'].rank(ascending=False, method='first')
        # 가중치 적용
        df['prediction'] *= weights[i]
        ranked_dfs.append(df)

    # 순위 및 가중치 적용된 데이터프레임들을 하나로 합치기
    combined_df = pd.concat(ranked_dfs, ignore_index=True)
    
    # resume_seq와 recruitment_seq를 기준으로 prediction 값을 그룹화하고 평균 계산
    averaged_predictions = combined_df.groupby(['resume_seq', 'recruitment_seq']).prediction.mean().reset_index()
    
    # 평균 순위를 기준으로 오름차순 정렬 (낮은 순위가 높은 선호도를 의미)
    top_k = (
        averaged_predictions
        .groupby('resume_seq')
        .apply(lambda x: x.nsmallest(final_K, 'prediction'))
        .reset_index(drop=True)
    )
    
    return top_k

In [6]:
# 함수 호출
recommendations = ensemble_recommendations('./preds', final_K=5)

# 결과 저장
recommendations[['resume_seq', 'recruitment_seq']].to_csv('submission_ensemble.csv', index=False)

# End